In [1]:
import glob
import logging
import os

import numpy as np
import whitebox
from joblib import Parallel, delayed

# Terrain conditioning using Whitebox Tools in parallel

An alternative to the PySheds workflow. Uses Whitebox Tools in parallel. Fast and seems promising, but at present PySheds is giving better results.

In [2]:
res = 40
n_jobs = 25

work_dir = r"/home/jovyan/shared/01_datasets/spatial"

In [3]:
%%time

wbt = whitebox.WhiteboxTools()
wbt.set_verbose_mode(True)
wbt.set_compress_rasters(True)
wbt.set_working_dir(work_dir)


def condition_dem_wbt(
    src_path,
    riv_path,
    burn_path,
    fill_path,
    fdir_path,
    facc_path,
    vassom,
):
    def log_callback(msg):
        logging.info(msg)

    log_file = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/wbt_logs/vassom_{vassom}_{res}m.log"
    logging.basicConfig(
        filename=log_file,
        format="%(asctime)s %(message)s",
        datefmt="%Y-%m-%d %H:%M",
        encoding="utf-8",
        filemode="w",
        level=logging.INFO,
    )

    #     wbt.fill_burn(
    #         src_path,
    #         riv_path,
    #         burn_path,
    #         callback=log_callback,
    #     )

    #     wbt.flow_accumulation_full_workflow(
    #         burn_path,
    #         fill_path,
    #         fdir_path,
    #         facc_path,
    #         out_type="cells",
    #         log=False,
    #         clip=False,
    #         esri_pntr=False,
    #         callback=log_callback,
    #     )

    wbt.fill_burn(
        src_path,
        riv_path,
        burn_path,
        callback=log_callback,
    )

    # msg = "    Filling depressions..."
    # print(msg)
    # logging.info(msg)
    # wbt.fill_depressions(
    #     burn_path,
    #     fill_path,
    #     fix_flats=True,
    #     flat_increment=None,
    #     max_depth=None,
    #     callback=log_callback,
    # )

    wbt.breach_depressions_least_cost(
        burn_path,
        fill_path,
        100000,
        max_cost=None,
        min_dist=True,
        flat_increment=None,
        fill=True,
        callback=log_callback,
    )

    wbt.d8_pointer(
        fill_path,
        fdir_path,
        esri_pntr=False,
        callback=log_callback,
    )

    wbt.d8_flow_accumulation(
        fdir_path,
        facc_path,
        out_type="cells",
        log=False,
        clip=False,
        pntr=True,
        esri_pntr=False,
        callback=log_callback,
    )


search_path = f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/dtm/*.tif"
flist = sorted(glob.glob(search_path))
vassom_list = [os.path.split(fname)[1].split("_")[1] for fname in flist]
riv_path = r"/home/jovyan/shared/01_datasets/spatial/temp/elvis_streams.shp"
burn_paths = [
    f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/wbt_burn/vassom_{vassom}_{res}m_burn_only.tif"
    for vassom in vassom_list
]
fill_paths = [
    f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/wbt_fill/vassom_{vassom}_{res}m_burn_fill.tif"
    for vassom in vassom_list
]
fdir_paths = [
    f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/wbt_fdir/vassom_{vassom}_{res}m_fdir.tif"
    for vassom in vassom_list
]
facc_paths = [
    f"/home/jovyan/shared/01_datasets/spatial/dtm_merged_utm33/dtm_{res}m/by_vassom/wbt_facc/vassom_{vassom}_{res}m_facc.tif"
    for vassom in vassom_list
]

Parallel(n_jobs=n_jobs)(
    delayed(condition_dem_wbt)(
        src_path,
        riv_path,
        burn_paths[idx],
        fill_paths[idx],
        fdir_paths[idx],
        facc_paths[idx],
        vassom_list[idx],
    )
    for idx, src_path in enumerate(flist)
)

['./whitebox_tools', '-v']
CPU times: user 669 ms, sys: 321 ms, total: 989 ms
Wall time: 6min 26s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,